# Métricas para sistemas de recomendación

En esta clase trabajaremos con un dataset de goodreads.

Vamos a tener 2 archivos en formato CSV. En uno tenemos metadata sobre libros y en otro interacciones entre usuarios y libros.

Los datasets deberán descargarlos de los siguientes links:

- https://www.kaggle.com/datasets/arashnic/goodreads-interactions-dataset/data
- https://mengtingwan.github.io/data/goodreads#datasets


In [ ]:
from google.colab import drive # La usamos para montar nuestra unidad de Google Drive
drive.mount('/content/drive') # Montamos nuestra unidad de Google Drive

Mounted at /content/drive


Cargamos los datasets (puede demorar unos minutos)

In [ ]:
import pandas as pd

In [ ]:
books_metadata = pd.read_csv('/content/drive/MyDrive/books_metadata_large.csv')
interactions = pd.read_csv('/content/drive/MyDrive/interactions_large.csv')
#books_metadata = pd.read_json('/content/drive/MyDrive/Programa Académico/Clases/Clase_42_Rec_sys_metricas/goodreads_books_poetry.json', lines=True)
#interactions = pd.read_json('/content/drive/MyDrive/Programa Académico/Clases/Clase_42_Rec_sys_metricas/goodreads_interactions_poetry.json', lines=True)

In [ ]:
books_metadata.head()

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series
0,NaN,1,[],US,eng,"[{'count': '8', 'name': 'to-read'}, {'count': ...",NaN,False,3.83,NaN,...,11.0,NaN,1887.0,https://www.goodreads.com/book/show/16037549-v...,https://images.gr-assets.com/books/1348176637m...,16037549,3,5212748,Vision of Sir Launfal and Other Poems,Vision of Sir Launfal and Other Poems
1,0811223981,2,[],US,NaN,"[{'count': '100', 'name': 'to-read'}, {'count'...",NaN,False,3.83,B00U2WY9U8,...,4.0,NaN,2015.0,https://www.goodreads.com/book/show/22466716-f...,https://images.gr-assets.com/books/1404958407m...,22466716,37,41905435,Fairy Tales: Dramolettes,Fairy Tales: Dramolettes
2,0374428115,7,[],US,NaN,"[{'count': '32', 'name': 'to-read'}, {'count':...",NaN,False,4.38,NaN,...,7.0,NaN,2008.0,https://www.goodreads.com/book/show/926662.Gro...,https://s.gr-assets.com/assets/nophoto/book/11...,926662,45,911665,Growltiger's Last Stand and Other Poems,Growltiger's Last Stand and Other Poems
3,0156182890,12,[],US,NaN,"[{'count': '554', 'name': 'to-read'}, {'count'...",NaN,False,3.71,B00IWTRB1W,...,3.0,NaN,1964.0,https://www.goodreads.com/book/show/926667.The...,https://images.gr-assets.com/books/1382939971m...,926667,115,995066,The Cocktail Party,The Cocktail Party
4,1942004192,4,[],US,eng,"[{'count': '228', 'name': 'to-read'}, {'count'...",NaN,False,5.00,NaN,...,12.0,First,2015.0,https://www.goodreads.com/book/show/29065952-l...,https://images.gr-assets.com/books/1455198396m...,29065952,9,49294781,Louder Than Everything You Love,Louder Than Everything You Love


In [ ]:
books_metadata.shape

(36514, 29)

In [ ]:
interactions.head()

,user_id,book_id,review_id,is_read,rating,review_text_incomplete,date_added,date_updated,read_at,started_at
0,8842281e1d1347389f2ab93d60773d4d,1384,1bad0122cebb4aa9213f9fe1aa281f66,True,4,NaN,Wed May 09 09:33:44 -0700 2007,Wed May 09 09:33:44 -0700 2007,NaN,NaN
1,8842281e1d1347389f2ab93d60773d4d,1376,eb6e502d0c04d57b43a5a02c21b64ab4,True,4,NaN,Wed May 09 09:33:18 -0700 2007,Wed May 09 09:33:18 -0700 2007,NaN,NaN
2,8842281e1d1347389f2ab93d60773d4d,30119,787564bef16cb1f43e0f641ab59d25b7,True,5,NaN,Sat Jan 13 13:44:20 -0800 2007,Wed Mar 22 11:45:08 -0700 2017,Tue Mar 01 00:00:00 -0800 1983,NaN
3,72fb0d0087d28c832f15776b0d936598,24769928,8c80ee74743d4b3b123dd1a2e0c0bcac,False,0,NaN,Wed Apr 27 11:05:51 -0700 2016,Wed Apr 27 11:05:52 -0700 2016,NaN,NaN
4,72fb0d0087d28c832f15776b0d936598,30119,2a83589fb597309934ec9b1db5876aaf,True,3,NaN,Mon Jun 04 18:58:08 -0700 2012,Mon Jun 04 18:58:13 -0700 2012,NaN,NaN


In [ ]:
interactions.shape

(2734350, 10)

En nuestro caso, como vamos a hacer un filtro colaborativo puro (sin basarnos en metadata para generar recomendaciones) solo utilizaremos un dataset: interacciones

Primero que nada seleccionamos las columnas que nos pueden ser útiles.

In [ ]:
interactions_selected = interactions[['user_id', 'book_id', 'is_read', 'rating', 'date_added']]

interactions_selected.head()

,user_id,book_id,is_read,rating,date_added
0,8842281e1d1347389f2ab93d60773d4d,1384,True,4,Wed May 09 09:33:44 -0700 2007
1,8842281e1d1347389f2ab93d60773d4d,1376,True,4,Wed May 09 09:33:18 -0700 2007
2,8842281e1d1347389f2ab93d60773d4d,30119,True,5,Sat Jan 13 13:44:20 -0800 2007
3,72fb0d0087d28c832f15776b0d936598,24769928,False,0,Wed Apr 27 11:05:51 -0700 2016
4,72fb0d0087d28c832f15776b0d936598,30119,True,3,Mon Jun 04 18:58:08 -0700 2012


In [ ]:
interactions_selected.user_id.nunique()

377799

In [ ]:
interactions_selected.book_id.nunique()

36514

Para que podamos correr el notebook con los recursos disponibles, limitaremos la cantidad de usuarios.

Vamos a seleccionar 20.000 usuarios random (no nos alcanza la ram que nos da colab para más que esto)

In [ ]:
import random
random.seed(0)

selected_users = random.sample(list(interactions_selected['user_id'].unique()), k=20000)

interactions_selected = interactions_selected[interactions_selected['user_id'].isin(selected_users)]

Ahora vamos a dividir en train y test basandonos en la columna date added.

Primero vamos a convertir la columna a tipo datetime

In [ ]:
%%time
interactions_selected["date_added"] = pd.to_datetime(interactions_selected.date_added, format="%a %b %d %H:%M:%S %z %Y")

CPU times: user 1.96 s, sys: 34 ms, total: 2 s
Wall time: 2.08 s


<timed exec>:1: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`


Removemos la hora y la zona horaria ya que no nos sirven en este caso

In [ ]:
interactions_selected["date_added"] = interactions_selected["date_added"].apply(lambda x: x.replace(tzinfo=None).date())

In [ ]:
interactions_selected.head()

,user_id,book_id,is_read,rating,date_added
39,96a4eae3201cf9eb0fd36a900611c925,23513349,True,5,2016-06-26
40,96a4eae3201cf9eb0fd36a900611c925,22151696,False,0,2015-10-27
41,96a4eae3201cf9eb0fd36a900611c925,11904233,False,0,2015-05-31
42,96a4eae3201cf9eb0fd36a900611c925,262304,False,0,2013-05-15
136,148cc5f044381ae63aacee1cd01d8ef1,5289,False,0,2012-06-11


Ahora nos fijamos en que rango de fechas tenemos interacciones

In [ ]:
interactions_selected.date_added.min()

datetime.date(2007, 2, 15)

In [ ]:
interactions_selected.date_added.max()

datetime.date(2017, 11, 3)

Para el dataset de train tomaremos las interacciones que ocurrieron hasta el día 1 de enero de 2016

In [ ]:
from datetime import datetime

In [ ]:
interactions_selected.shape

(146825, 5)

In [ ]:
interactions_train = interactions_selected[interactions_selected.date_added < datetime(2016, 1, 1).date()]

interactions_test = interactions_selected[interactions_selected.date_added >= datetime(2016, 1, 1).date()]

In [ ]:
interactions_train.shape

(100102, 5)

In [ ]:
interactions_test.shape

(46723, 5)

Ya estamos en condiciones de armar la matriz de interacciones

In [ ]:
user_book_interaction = pd.pivot_table(interactions_train[["user_id", "book_id", "rating"]], index='user_id', columns='book_id', values='rating')

# Completamos nulos con 0
user_book_interaction = user_book_interaction.fillna(0)

user_book_interaction.sample(10)

book_id,234,236,244,254,284,285,286,289,290,291,...,29363959,29589253,29884630,29925028,30638695,31421949,33614551,34023590,34031517,35612585
user_id,,,,,,,,,,,,,,,,,,,,,
f2116dbc08dd31eccb9971f2b2bc7f57,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21dc28bc49b347eff0a39773b7d99fb0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fe2b650b818ceb4d1e4f57087b3b93f7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39b79e93103f5262d7ae3a6ba4dd2251,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
f4b646bc322e264b40c02c784494b682,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
bad5c5830565fab9417520ee45fb0ae9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3d3801216f146019562de5ca30542f7e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53fb3bbd76768d37b3ec519a093e5a6a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
f6997aa934441603dd2cd7255e41c151,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
user_book_interaction.shape

(15125, 15569)

Y el diccionario para futura referencia:

In [ ]:
user_id = list(user_book_interaction.index)
user_dict = {}
counter = 0
for i in user_id:
    user_dict[i] = counter
    counter += 1

In [ ]:
item_id = list(user_book_interaction.columns)
item_dict = {}
counter = 0
for i in item_id:
    item_dict[i] = counter
    counter += 1

Convertimos a csr matrix:

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
user_book_interaction_csr = csr_matrix(user_book_interaction.values)
user_book_interaction_csr

<15125x15569 sparse matrix of type '<class 'numpy.float64'>'
	with 51469 stored elements in Compressed Sparse Row format>

Entrenamos el modelo:

In [ ]:
pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808325 sha256=df5a5d94353e83872bbc2a2000f256e8ee5cb22c3a07fffbfa05806237c49fe9
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [ ]:
from lightfm import LightFM

In [ ]:
model = LightFM(random_state=0,
                loss='warp',
                learning_rate=0.03,
                no_components=100)

model = model.fit(user_book_interaction_csr,
                  epochs=100,
                  num_threads=16, verbose=False)

Y ahora definimos la funcion para obtener recomendaciones para un usuario en específico:

In [ ]:
import numpy as np

In [ ]:
def recomm(model, interactions, user_id, user_dict, item_dict, nrec_items = 10, show = True):

    n_users, n_items = interactions.shape
    user_x = user_dict[user_id]

    scores = pd.Series(model.predict(user_x,np.arange(n_items)))
    scores.index = interactions.columns
    scores = list(pd.Series(scores.sort_values(ascending=False).index))

    known_items = list(pd.Series(interactions.loc[user_id,:] \
                                [interactions.loc[user_id,:] > 0].index).sort_values(ascending=False))

    #Filtro de items que ya leyó el usuario
    scores = [x for x in scores if x not in known_items]
    recomms = scores[0:nrec_items]

    if show == True:
        print ("User: " + str(user_id))
        print("Known Likes:")
        counter = 1
        for i in known_items:
            print(str(counter) + '- ' + str(i))
            counter+=1

        print("\n Recommended Items:")

        counter = 1
        for i in recomms:
            print(str(counter) + '- ' + str(i))
            counter+=1
    return recomms

Probamos la función para un usuario al azar. Al poner la opción show=True nos imprime los known likes y recomms.



In [ ]:
recomms_test = recomm(model, user_book_interaction, '646b91757d6c3b9460273521acce77d5', user_dict, item_dict, show=True)

User: 646b91757d6c3b9460273521acce77d5
Known Likes:
1- 1352562
2- 1277995
3- 1277014
4- 752798
5- 726121
6- 641861
7- 46983

 Recommended Items:
1- 3212085
2- 1073100
3- 16038843
4- 2023749
5- 22593508
6- 207354
7- 20839571
8- 21028697
9- 1524722
10- 13131601


In [ ]:
recomms_test

[3212085,
 1073100,
 16038843,
 2023749,
 22593508,
 207354,
 20839571,
 21028697,
 1524722,
 13131601]

Si queremos ver el nombre de estos libros, podemos buscar en el dataset de metadata

In [ ]:
books_metadata.loc[books_metadata.book_id.isin(recomms_test), "title"]

,title
784,Mean Time
3687,Hunting Season
8215,A Boy's Will
11726,Mình Phải Sống Như Mùa Hè Năm Ấy
11876,The Rose That Grew From Concrete
14239,El libro de los abrazos
17741,Les Fleurs du Mal
18704,Shiva's Arms
29864,Skylight
32645,Libro del desasosiego


# Métricas

Ahora para medir métricas, necesitamos:

1. Generar recomendaciones para todos los usuarios
2. Comparar lo que recomendamos contra lo que los usuarios realmente vieron (set de test).


Comencemos generando recomendaciones para todos los usuarios.

Nuestra función (recomm) sirve para generar recomendaciones personalizadas a usuarios para los cuales tenemos datos en el set de train.

Para usuarios cold start (que no aparecen en el set de train), debemos buscar alguna alternativa. En este caso les recomendaremos los libros con rating promedio más alto.

In [ ]:
cold_start_recomms = interactions_train\
                      .groupby("book_id")\
                      .agg({"rating": "mean"})\
                      .sort_values(by="rating", ascending=False)\
                      .head(10)\
                      .index\
                      .to_list()

A todos los usuarios cold start les recomendaremos:

In [ ]:
cold_start_recomms

[2856684,
 3387595,
 3223648,
 3229446,
 3229707,
 3234022,
 162337,
 3237379,
 3271021,
 3277923]

Ahora generemos recomendaciones para todos los usuarios.

Primero comenzamos armando un listado de usuarios "non cold" a los cuales les daremos recomms personalizadas.

In [ ]:
users = interactions_selected.user_id.unique()
non_cold_users = interactions_train.user_id.unique()

print(len(users), len(non_cold_users))

20000 15125


In [ ]:
recomms_dict = {
    "user": [],
    "recomms": [],
    "cold": []
}

for user in users:

  # Recomms personalizadas
  if user in non_cold_users:
    user_recomms = recomm(model, user_book_interaction, user, user_dict, item_dict, nrec_items = 10, show = False)
    recomms_dict["user"].append(user)
    recomms_dict["recomms"].append(user_recomms)
    recomms_dict["cold"].append(False) #Para luego tener identificados a los usuarios cold

  # Recomms cold
  else:
    recomms_dict["user"].append(user)
    recomms_dict["recomms"].append(cold_start_recomms)
    recomms_dict["cold"].append(True) #Para luego tener identificados a los usuarios cold

Ahora tenemos un diccionario de recomedaciones. Lo vamos a convertir en dataframe para poder manipularlo más fácil (como estamos acostumbrados)

In [ ]:
recomms_df = pd.DataFrame(recomms_dict)

In [ ]:
recomms_df.head()

,user,recomms,cold
0,96a4eae3201cf9eb0fd36a900611c925,"[30119, 1420, 1381, 30118, 1371, 2547, 23919, ...",False
1,148cc5f044381ae63aacee1cd01d8ef1,"[30119, 1420, 1381, 30118, 1371, 2547, 23919, ...",False
2,b54c83f02c748fe75ed309e1e12eace2,"[10691701, 6054132, 10243089, 5886412, 1887955...",False
3,8d88702ebbcd8b4d4e24ee13cb6a1d80,"[1420, 1381, 1371, 30119, 6656, 2696, 15645, 2...",False
4,7e4945a6266566e39e3f251fff0fdde1,"[30119, 1420, 1381, 23919, 30118, 2696, 1371, ...",False


In [ ]:
recomms_df.user.nunique()

20000

In [ ]:
recomms_df.cold.value_counts()

,count
cold,
False,15125
True,4875


Ahora tenemos que comparar nuestras recomms contra lo que los usuarios realmente vieron (test).



In [ ]:
interactions_test.head()

,user_id,book_id,is_read,rating,date_added
39,96a4eae3201cf9eb0fd36a900611c925,23513349,True,5,2016-06-26
332,7e4945a6266566e39e3f251fff0fdde1,25877517,False,0,2017-08-29
333,7e4945a6266566e39e3f251fff0fdde1,1211497,False,0,2017-05-29
334,7e4945a6266566e39e3f251fff0fdde1,576022,False,0,2017-04-24
335,7e4945a6266566e39e3f251fff0fdde1,49461,False,0,2016-07-23


Primero ordenamos libros según los ratings (mayor a menor) que dieron los usuarios en el set de test.

Luego, agrupamos y armamos un listado de libros para cada usuario.

Este es el listado "ideal" contra el que vamos a comparar nuestras recomendaciones.

In [ ]:
ideal_recomms = interactions_test[interactions_test.is_read]\
                  .sort_values(by=["user_id", "rating"], ascending=False)\
                  .groupby(["user_id"], as_index=False)\
                  .agg({"book_id": "unique"})\
                  .head()
ideal_recomms.head()

,user_id,book_id
0,001cc735059908f79fd9b698b60514d0,"[2099146, 17618007]"
1,0023ccd9fffc2e46c88c46fe59fbc58a,[30718287]
2,0028dc19074f9caa90d4b6a44b7a39d6,[32334098]
3,0036e193ef216f74d150e31c5b2445bc,[15645]
4,003c3cbe1f0bf247fc1bae43984e333b,[33911858]


Ahora vamos a medir MAP. Para esto nos basaremos en este ejemplo: https://github.com/Datathon2021/Recomendador/blob/main/Ejemplos/ejemplo_map.ipynb

Sklearn también tiene funciones para medir NDCG, etc.


Primero que nada, vamos a unir en un mismo dataframe las recomendaciones nuestras con el set ideal.

In [ ]:
df_map = ideal_recomms.merge(recomms_df, how="left", left_on="user_id", right_on="user")[["user_id", "book_id", "recomms"]]
df_map.columns = ["user_id", "ideal", "recomms"]
df_map.head()

,user_id,ideal,recomms
0,001cc735059908f79fd9b698b60514d0,"[2099146, 17618007]","[1420, 1381, 1371, 30119, 6656, 15997, 12914, ..."
1,0023ccd9fffc2e46c88c46fe59fbc58a,[30718287],"[2856684, 3387595, 3223648, 3229446, 3229707, ..."
2,0028dc19074f9caa90d4b6a44b7a39d6,[32334098],"[1420, 23919, 395090, 1432, 5932, 2696, 76889,..."
3,0036e193ef216f74d150e31c5b2445bc,[15645],"[30119, 1420, 1381, 30118, 1371, 2547, 23919, ..."
4,003c3cbe1f0bf247fc1bae43984e333b,[33911858],"[2856684, 3387595, 3223648, 3229446, 3229707, ..."


In [ ]:
aps = [] # lista vacía para ir almacenando la AP de cada recomendación

for pred, label in df_map[["ideal", "recomms"]].values:
  n = len(pred) # cantidad de elementos recomendados
  arange = np.arange(n, dtype=np.int32) + 1. # indexamos en base 1
  rel_k = np.in1d(pred[:n], label) # lista de booleanos que indican la relevancia de cada ítem
  tp = np.ones(rel_k.sum(), dtype=np.int32).cumsum() # lista con el contador de verdaderos positivos
  denom = arange[rel_k] # posiciones donde se encuentran los ítems relantes
  ap = (tp / denom).sum() / len(label) # average precision
  aps.append(ap)

In [ ]:
MAP = np.mean(aps)
print(f'mean average precision = {round(MAP, 5)}')

mean average precision = 0.02
